In [ ]:
!pip -q install earthengine-api pandas openpyxl tqdm

# 중간 랜덤샘플링 데이터셋 기준 NDVI 추출 코드

## 1차 NDVI 저장

In [ ]:
import ee, pandas as pd, numpy as np, os, math, traceback, time
from datetime import datetime, timedelta
from tqdm import tqdm


try:
    ee.Initialize(project='matprocject11')
    print("GEE initialized")
except Exception as e:
    print("Init failed, authenticating...", repr(e))
    ee.Authenticate()
    ee.Initialize()
    print("GEE authenticated & initialized")

# ---------------- 경로/컬럼 설정 ----------------
INPUT_PATH   = "/content/TOTAL_DWI_CatForest_cleaned.xlsx"  # Colab에 업로드한 파일 경로
OUTPUT_PATH  = "/content/TOTAL_DWI_with_NDVI11111.xlsx"
CHECKPOINT   = "/content/_ndvi_checkpoint.xlsx"
LOG_PATH     = "/content/_ndvi_log.txt"

DATE_COL, LAT_COL, LON_COL = "date", "lat", "lon"


LANDSAT_ID = "LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI"
MODIS_ID   = "MODIS/061/MOD13Q1"
BAND       = "NDVI"


WINDOW_STEPS = [16, 24, 32, 45, 60]
BUFFER_STEPS = [120, 300, 600, 1000]

SAVE_EVERY   = 100
PRINT_EVERY  = 50


def _closest_image(col, target_date_str: str) -> ee.Image:

    d0 = ee.Date(target_date_str)
    def add_diff(img):
        t = ee.Date(img.get('system:time_start'))
        return img.set({
            'date_diff': t.difference(d0, 'day').abs(),
            'img_date': t.format('YYYY-MM-dd')
        })
    return ee.Image(col.map(add_diff).sort('date_diff').first())


def _try_dataset(lat: float, lon: float, date_str: str,
                 dataset_id: str, band: str, is_modis: bool):

    point = ee.Geometry.Point([float(lon), float(lat)])
    scale = 250 if dataset_id == MODIS_ID else 30
    sfac  = 0.0001 if is_modis else 1.0


    for win in WINDOW_STEPS:
        start = ee.Date((datetime.strptime(date_str, "%Y-%m-%d") - timedelta(days=win)).strftime("%Y-%m-%d"))
        end   = ee.Date((datetime.strptime(date_str, "%Y-%m-%d") + timedelta(days=win+1)).strftime("%Y-%m-%d"))
        col = ee.ImageCollection(dataset_id).filterDate(start, end).select(band)
        if col.size().getInfo() == 0:
            continue

        img = _closest_image(col, date_str)
        used_date = img.get('img_date').getInfo()


        for buf in BUFFER_STEPS:
            geom = point.buffer(buf)
            try:
                d = img.reduceRegion(
                    reducer=ee.Reducer.median(),
                    geometry=geom, scale=scale,
                    bestEffort=True, maxPixels=1e8
                ).get(band).getInfo()
            except Exception:
                d = None
            if d is not None:
                v = max(-1.0, min(1.0, float(d) * sfac))
                return v, used_date, win, buf, dataset_id


        try:
            fc = img.sample({
                'region': point, 'scale': scale,
                'numPixels': 1, 'geometries': False, 'tileScale': 2
            })
            d2 = None if fc.size().getInfo() == 0 else ee.Feature(fc.first()).get(band).getInfo()
        except Exception:
            d2 = None
        if d2 is not None:
            v2 = max(-1.0, min(1.0, float(d2) * sfac))
            return v2, used_date, win, 0, dataset_id


        filled = img.select(band).unmask().focal_mean(radius=150, units='meters', iterations=1)
        for buf2 in BUFFER_STEPS:
            geom2 = point.buffer(buf2)
            try:
                d3 = filled.reduceRegion(
                    reducer=ee.Reducer.median(),
                    geometry=geom2, scale=scale,
                    bestEffort=True, maxPixels=1e8
                ).get(band).getInfo()
            except Exception:
                d3 = None
            if d3 is not None:
                v3 = max(-1.0, min(1.0, float(d3) * sfac))
                return v3, used_date, win, buf2, dataset_id

    return None, None, None, None, None


def get_ndvi(lat: float, lon: float, date_str: str):

    v, used, win, buf, src = _try_dataset(lat, lon, date_str, LANDSAT_ID, BAND, False)
    if v is not None:
        return v, used, os.path.basename(src), win, buf

    v, used, win, buf, src = _try_dataset(lat, lon, date_str, MODIS_ID, BAND, True)
    if v is not None:
        return v, used, os.path.basename(src), win, buf
    return np.nan, None, None, None, None


df = pd.read_excel(INPUT_PATH)

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce").dt.strftime("%Y-%m-%d")


for c in ["NDVI", "NDVI_used_date", "NDVI_source", "NDVI_window", "NDVI_buffer"]:
    if c not in df.columns:
        df[c] = np.nan if c == "NDVI" else None


errors = []
filled = 0

for i, row in tqdm(df.iterrows(), total=len(df)):
    lat, lon, d = row[LAT_COL], row[LON_COL], row[DATE_COL]

    if pd.isna(lat) or pd.isna(lon) or pd.isna(d):
        df.at[i, "NDVI"]            = np.nan
        df.at[i, "NDVI_used_date"]  = None
        df.at[i, "NDVI_source"]     = None
        df.at[i, "NDVI_window"]     = None
        df.at[i, "NDVI_buffer"]     = None
        continue

    try:
        ndvi, used_date, src, win, buf = get_ndvi(float(lat), float(lon), str(d))
        df.at[i, "NDVI"]            = ndvi
        df.at[i, "NDVI_used_date"]  = used_date
        df.at[i, "NDVI_source"]     = src
        df.at[i, "NDVI_window"]     = win
        df.at[i, "NDVI_buffer"]     = buf
        if not (pd.isna(ndvi) or (isinstance(ndvi, float) and math.isnan(ndvi))):
            filled += 1
    except Exception as e:
        errors.append(f"[row {i}] {type(e).__name__}: {repr(e)}")
        df.at[i, "NDVI"]            = np.nan
        df.at[i, "NDVI_used_date"]  = None
        df.at[i, "NDVI_source"]     = None
        df.at[i, "NDVI_window"]     = None
        df.at[i, "NDVI_buffer"]     = None


    if (i + 1) % PRINT_EVERY == 0:
        print(f"[{i+1}/{len(df)}] filled so far: {filled}")
    if (i + 1) % SAVE_EVERY == 0:
        df.to_excel(CHECKPOINT, index=False)
        print("Checkpoint saved ->", CHECKPOINT)


df = df.sort_values([LAT_COL, LON_COL, DATE_COL])
def _fill_group(g):
    g["NDVI"] = g["NDVI"].astype(float)
    g["NDVI"] = g["NDVI"].interpolate(method="linear", limit_direction="both")
    return g
df = df.groupby([LAT_COL, LON_COL], group_keys=False).apply(_fill_group)


df.to_excel(OUTPUT_PATH, index=False)
print("Saved ->", OUTPUT_PATH, "| rows:", len(df), "| filled:", int((~pd.isna(df['NDVI'])).sum()))

if errors:
    with open(LOG_PATH, "w", encoding="utf-8") as f:
        f.write("\n".join(errors))
    print(f"Logged {len(errors)} exceptions -> {LOG_PATH}")
else:
    print("No exceptions.")


GEE initialized


 11%|█         | 50/446 [00:55<04:10,  1.58it/s]

[50/446] filled so far: 50


 22%|██▏       | 99/446 [02:22<08:26,  1.46s/it]

[100/446] filled so far: 100


...

 90%|████████▉ | 400/446 [12:28<00:40,  1.14it/s]

[400/446] filled so far: 400
Checkpoint saved -> /content/_ndvi_checkpoint.xlsx


100%|██████████| 446/446 [13:24<00:00,  1.80s/it]


✅ Saved -> /content/TOTAL_DWI_with_NDVI11111.xlsx | rows: 446 | filled: 446
No exceptions.


/tmp/ipython-input-2055250588.py:188: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby([LAT_COL, LON_COL], group_keys=False).apply(_fill_group)


## 2차 NDVI 저장 (0으로 저장된 결측치 채우기)

In [ ]:

try:
    ee.Initialize(project='matprocject11')
    print("GEE initialized")
except Exception:
    ee.Authenticate()
    ee.Initialize()
    print("GEE authenticated & initialized")


INPUT_XLSX  = "/content/TOTAL_DWI_with_NDVI11111.xlsx"
OUTPUT_XLSX = "/content/TOTAL_DWI_with_NDVI11111_filled.xlsx"

DATE_COL = None
LON_COL  = None   '
LAT_COL  = None
NDVI_COL = None

BUF_METERS   = 120
LS_WIN_DAYS  = 32
S2_WIN_DAYS  = 20
CLIM_YEARS   = 5
S2_CLOUD_TH  = 40
REDUCER      = ee.Reducer.mean()


def _safe_date(s):
    if pd.isna(s): return None
    if isinstance(s, (pd.Timestamp, datetime)):
        return pd.to_datetime(s).strftime("%Y-%m-%d")
    try:
        return pd.to_datetime(str(s)).strftime("%Y-%m-%d")
    except Exception:
        return None

def _value_at(img, band, geom, scale, reducer=REDUCER):
    d = img.select(band).reduceRegion(
        reducer=reducer, geometry=geom, scale=scale,
        maxPixels=1e8, bestEffort=True
    )
    return d.get(band)

def _closest(col, target_date):
    t = ee.Date(target_date)
    col2 = col.map(lambda im: im.set('d', ee.Date(im.get('system:time_start')).difference(t,'day').abs()))
    return ee.Image(col2.sort('d').first())

def _landsat_ndvi(point, date_str):
    t  = ee.Date(date_str)
    st = t.advance(-LS_WIN_DAYS,'day')
    en = t.advance( LS_WIN_DAYS,'day').advance(1,'day')

    col = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
           .merge(ee.ImageCollection('LANDSAT/LC09/C02/T1_L2'))
           .filterDate(st, en)
           .filterBounds(point))

    def _mask_map(img):
        qa = img.select('QA_PIXEL')
        cloud  = qa.bitwiseAnd(1<<3).neq(0)
        shadow = qa.bitwiseAnd(1<<4).neq(0)
        snow   = qa.bitwiseAnd(1<<5).neq(0)
        mask = cloud.Or(shadow).Or(snow).Not()
        sr = img.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']).multiply(0.0000275).add(-0.2)
        nir = sr.select('SR_B5')
        red = sr.select('SR_B4')
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI').updateMask(mask)
        return ndvi.copyProperties(img, img.propertyNames())

    ndvi_col = col.map(_mask_map)
    if ndvi_col.size().getInfo() == 0:
        return None, None
    best = _closest(ndvi_col, t)
    geom = point if BUF_METERS==0 else point.buffer(BUF_METERS)
    val  = _value_at(best, 'NDVI', geom, 30)
    return val, ee.Date(best.get('system:time_start')).format('YYYY-MM-dd')

def _s2_ndvi(point, date_str):
    t  = ee.Date(date_str)
    st = t.advance(-S2_WIN_DAYS,'day')
    en = t.advance( S2_WIN_DAYS,'day').advance(1,'day')

    s2  = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate(st, en).filterBounds(point))

    prob = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY').filterDate(st, en).filterBounds(point)

    joined = ee.Join.saveFirst('cloud_prob').apply(
        primary=s2,
        secondary=prob,
        condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
    )

    def _map(img):
        img = ee.Image(img)
        cp  = ee.Image(img.get('cloud_prob'))
        cp  = ee.Algorithms.If(cp, ee.Image(cp).select('probability'), ee.Image(0).rename('probability'))
        cp  = ee.Image(cp)

        scl = img.select('SCL')
        cloudMask = cp.gt(S2_CLOUD_TH)\
            .Or(scl.eq(3)).Or(scl.eq(8)).Or(scl.eq(9)).Or(scl.eq(10)).Or(scl.eq(11)).Not()

        b4 = img.select('B4').multiply(0.0001)
        b8 = img.select('B8').multiply(0.0001)
        ndvi = b8.subtract(b4).divide(b8.add(b4)).rename('NDVI').updateMask(cloudMask)
        return ndvi.copyProperties(img, img.propertyNames())

    ndvi_col = ee.ImageCollection(joined).map(_map)
    if ndvi_col.size().getInfo() == 0:
        return None, None
    best = _closest(ndvi_col, t)
    geom = point if BUF_METERS==0 else point.buffer(BUF_METERS)
    val  = _value_at(best, 'NDVI', geom, 10)
    return val, ee.Date(best.get('system:time_start')).format('YYYY-MM-dd')

def _modis_ndvi(point, date_str):
    t = ee.Date(date_str)
    col = (ee.ImageCollection('MODIS/061/MOD13Q1')
           .filterDate(t.advance(-40,'day'), t.advance(40,'day'))
           .filterBounds(point)
           .select('NDVI'))

    if col.size().getInfo() == 0:
        return None, None
    best = _closest(col, t)
    ndvi = ee.Image(best).multiply(0.0001).rename('NDVI')
    geom = point if BUF_METERS==0 else point.buffer(BUF_METERS)
    val  = _value_at(ndvi, 'NDVI', geom, 250)
    return val, ee.Date(best.get('system:time_start')).format('YYYY-MM-dd')

def _modis_monthly_clim(point, date_str):
    t = ee.Date(date_str)
    month = t.get('month')
    start = t.advance(-CLIM_YEARS, 'year')
    col = (ee.ImageCollection('MODIS/061/MOD13Q1')
           .filterDate(start, t)
           .filterBounds(point)
           .filter(ee.Filter.calendarRange(month, month, 'month'))
           .select('NDVI')
           .map(lambda im: ee.Image(im).multiply(0.0001).rename('NDVI')))
    if col.size().getInfo() == 0:
        return None, None
    clim = col.median()
    geom = point if BUF_METERS==0 else point.buffer(BUF_METERS)
    val  = _value_at(clim, 'NDVI', geom, 250)
    return val, ee.String('climatology(m=').cat(ee.Number(month).format()).cat(')')

def get_ndvi_with_fallback(lat, lon, date_str):
    pt = ee.Geometry.Point([float(lon), float(lat)])
    funcs = [_landsat_ndvi, _s2_ndvi, _modis_ndvi, _modis_monthly_clim]
    for f in funcs:
        try:
            val, when = f(pt, date_str)
            if val is None:
                continue
            is_null = ee.Algorithms.IsEqual(val, None).getInfo()
            if not is_null:
                return float(ee.Number(val).getInfo())
        except ee.EEException as e:
            time.sleep(0.5)
            continue
        except Exception:
            continue
    return None

df = pd.read_excel(INPUT_XLSX)
cols_lower = {c: c.lower() for c in df.columns}

def _guess(colnames, keys):
    for c in colnames:
        cl = c.lower()
        if any(k in cl for k in keys):
            return c
    return None

ndvi_col = NDVI_COL or _guess(df.columns, ['ndvi'])
date_col = DATE_COL or _guess(df.columns, ['date','날짜','datetime'])
lon_col  = LON_COL  or _guess(df.columns, ['lon','경도','x'])
lat_col  = LAT_COL  or _guess(df.columns, ['lat','위도','y'])

assert ndvi_col is not None, "NDVI 컬럼을 찾지 못했습니다. NDVI_COL에 명시하세요."
assert date_col is not None, "date 컬럼을 찾지 못했습니다. DATE_COL에 명시하세요."
assert lon_col  is not None and lat_col is not None, "lon/lat 컬럼을 찾지 못했습니다."

print("Detected columns ->",
      f"NDVI:{ndvi_col}, date:{date_col}, lon:{lon_col}, lat:{lat_col}")


mask = (df[ndvi_col].fillna(0) == 0) & df[lon_col].notna() & df[lat_col].notna() & df[date_col].notna()
rows_to_fill = df[mask].copy()
print(f"Rows to fill (NDVI==0): {len(rows_to_fill)} / {len(df)}")


filled_values = []
idx_list = rows_to_fill.index.tolist()

for i in tqdm(idx_list, desc="Filling NDVI"):
    lat = df.at[i, lat_col]
    lon = df.at[i, lon_col]
    date_str = _safe_date(df.at[i, date_col])
    if (lat is None) or (lon is None) or (date_str is None):
        filled_values.append((i, None))
        continue
    try:
        val = get_ndvi_with_fallback(lat, lon, date_str)
    except Exception:
        val = None
    filled_values.append((i, val))


for i, val in filled_values:
    if val is not None:
        df.at[i, ndvi_col] = val
    else:
        df.at[i, ndvi_col] = np.nan


df.to_excel(OUTPUT_XLSX, index=False)
print("Saved:", OUTPUT_XLSX)


# 고정좌표(15개) 기준 NDVI 추출 코드


In [ ]:
# -*- coding: utf-8 -*-
import ee
import pandas as pd
import numpy as np
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# GEE 초기화
try:
    ee.Initialize(project='matprocject11')
except:
    ee.Authenticate()
    ee.Initialize()

# 경로 설정
INPUT_CSV  = "/content/wildfire_dataset.csv"
OUTPUT_CSV = "/content/wildfire_dataset_with_NDVI.csv"

# 데이터 로드
df = pd.read_csv(INPUT_CSV)
df = df.rename(columns=str.lower)
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")

if "lat" not in df.columns or "lon" not in df.columns:
    raise ValueError("lat, lon, date 컬럼 필요")

# GEE 컬렉션 설정
LAND8_ID = "LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI"
MODIS_ID = "MODIS/061/MOD13Q1"

LAND8_COL = ee.ImageCollection(LAND8_ID)
MODIS_COL = ee.ImageCollection(MODIS_ID)

LAND8_SCALE = 30
MODIS_SCALE = 250
SPATIAL_LAND8 = [120, 250]
SPATIAL_MODIS = [250, 500]

# 가장 가까운 날짜 영상 선택
def closest_image(col, date_str):
    d0 = ee.Date(date_str)

    def add_diff(im):
        t = ee.Date(im.get("system:time_start"))
        return im.set("d", t.difference(d0, "day").abs())

    return ee.Image(col.map(add_diff).sort("d").first())

# 단일 좌표 NDVI 추출
def extract_ndvi_for_one(lat, lon, date_str):
    point = ee.Geometry.Point([lon, lat])

    # Landsat8
    try:
        img_l8 = closest_image(LAND8_COL, date_str).select("NDVI")
        for buf in SPATIAL_LAND8:
            v = img_l8.reduceRegion(
                reducer=ee.Reducer.median(),
                geometry=point.buffer(buf),
                scale=LAND8_SCALE,
                bestEffort=True
            ).get("NDVI").getInfo()
            if v is not None:
                return float(v)
    except:
        pass

    # MODIS
    try:
        img_m = closest_image(MODIS_COL, date_str).select("NDVI").multiply(0.0001)
        for buf in SPATIAL_MODIS:
            v = img_m.reduceRegion(
                reducer=ee.Reducer.median(),
                geometry=point.buffer(buf),
                scale=MODIS_SCALE,
                bestEffort=True
            ).get("NDVI").getInfo()
            if v is not None:
                return float(v)
    except:
        pass

    return None

# 병렬 처리
def worker(row):
    try:
        return extract_ndvi_for_one(row["lat"], row["lon"], row["date"])
    except:
        return None

rows = df.to_dict("records")
results = []

print("Extracting NDVI ...")

with ThreadPoolExecutor(max_workers=6) as ex:
    futures = {ex.submit(worker, r): i for i, r in enumerate(rows)}
    for f in tqdm(as_completed(futures), total=len(futures)):
        results.append(f.result())

# 결과 저장
df["NDVI"] = results
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print("Saved:", OUTPUT_CSV)
print("Shape:", df.shape)
print("NDVI non-null:", df["NDVI"].notnull().sum())


GEE initialized
Extracting NDVI from GEE ...


100%|██████████| 1463/1463 [1:02:22<00:00,  2.56s/it]

Saved: /content/wildfire_dataset_with_NDVI.csv
Shape: (1463, 28)
NDVI non-null: 1463
